<a href="https://colab.research.google.com/github/omtiwari28/Music_Generation/blob/main/Music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet_ml > /dev/null 2>&1
import comet_ml
# Replace with your actual Comet API key
comet_api_key = "yd6X45GKLWoIKspb4dChTuwOs"
import torch
import torch.optim as optim
import torch.nn as nn

!pip install mitdeeplearning --quiet
import mitdeeplearning as mdl
import numpy as np
import time , os , functools
from IPython import display
from tqdm import tqdm
from scipy.io.wavfile import write
import pygame
!apt-get install abcmidi timidity > /dev/null 2>&1
# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


In [ ]:
songs = mdl.lab1.load_training_data()

exa = songs[3]
print(exa)

In [ ]:
mdl.lab1.play_song(exa)

In [ ]:
songs_joined = "\n\n".join(songs)

# Find all unique characters in the joined string
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")

In [ ]:
char2idx = {u: i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)

In [ ]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
### Vectorize the songs string ###

'''TODO: Write a function to convert the all songs string to a vectorized
    (i.e., numeric) representation. Use the appropriate mapping
    above to convert from vocab characters to the corresponding indices.

  NOTE: the output of the `vectorize_string` function
  should be a np.array with `N` elements, where `N` is
  the number of characters in the input string
'''
songs_joined = "\n\n".join(songs)
def vectorize_string(string):
  vectorize_string = np.array([char2idx[char] for char in string])
  return vectorize_string

vectorized_songs = vectorize_string(songs_joined)

In [ ]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

In [ ]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):
    # the length of the vectorized songs string
    n = vectorized_songs.shape[0] - 1
    # randomly choose the starting indices for the examples in the training batch
    idx = np.random.choice(n - seq_length, batch_size)

    '''TODO: construct a list of input sequences for the training batch'''
    input_batch = [vectorized_songs[i : i +seq_length] for i in idx]

    '''TODO: construct a list of output sequences for the training batch'''
    output_batch = [vectorized_songs[i+1 : i + seq_length +1] for i in idx]

    # Convert the input and output batches to tensors
    x_batch = torch.tensor(input_batch, dtype=torch.long)
    y_batch = torch.tensor(output_batch, dtype=torch.long)

    return x_batch, y_batch

# Perform some simple tests to make sure your batch function is working properly!
test_args = (vectorized_songs, 10, 2)
x_batch, y_batch = get_batch(*test_args)
assert x_batch.shape == (2, 10), "x_batch shape is incorrect"
assert y_batch.shape == (2, 10), "y_batch shape is incorrect"
print("Batch function works correctly!")


In [ ]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(x_batch[0], y_batch[0])):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx.item()])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx.item()])))

Defining the RNN Model

In [ ]:
# defining the model
class lstmmodel(nn.Module):
    def __init__(self,vocab_size,embedding_dim , hidden_size):
        super(lstmmodel ,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size , embedding_dim)

        self.lstm = nn.LSTM(embedding_dim , hidden_size , batch_first = True)

        self.fc = nn.Linear(hidden_size , vocab_size)

    def init_hidden (self,batch_size,device) :
        #initialize both hidden state and cell state
        return (torch.zeros(1,batch_size,self.hidden_size).to(device),
                torch.zeros(1,batch_size,self.hidden_size).to(device))
    def forward(self, x , state = None , return_state = False ):
        x = self.embedding(x)

        if state is None :
            state = self.init_hidden(x.size(0) , x.device)
        out,state = self.lstm(x , state)

        out = self.fc(out)
        return out if not return_state else (out,state)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 1024
batch_size = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = lstmmodel(vocab_size,embedding_dim,hidden_size).to(device)

print(model)

#### Testing the data

In [ ]:
x , y = get_batch(vectorized_songs , seq_length=100 , batch_size=32)
x = x.to(device)
y = y.to(device)

pred = model(x)
print(f"Input shape :  {x.shape} ")
print(f"prediction shape :   {pred.shape}")

In [ ]:
sampled_indices = torch.multinomial(torch.softmax(pred[0], dim=-1), num_samples=1)
sampled_indices = sampled_indices.squeeze(-1).cpu().numpy()
sampled_indices

In [ ]:
print("Input: \n", repr("".join(idx2char[x[0].cpu()])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

#### Training the model and Loss checking operation

#### Defining the Lossy Function

In [ ]:
### Defining The Lossy Function ###
Cross_entropy = nn.CrossEntropyLoss()
def compute_loss(labels , logits ):
    """
        Inputs :
            labels: (batch_size , sequence_length)
            logits: (batch_size , sequence_length , vocab_size)

        output :
            loss : scalar cross entropy loss over the batch and sequence length
        """

    batched_labels = labels.view(-1)
    batched_logits = logits.view(-1 , logits.size(-1))

    loss = Cross_entropy(batched_logits , batched_labels)
    return loss

In [ ]:
### Compute the loss on the prediction from the untrained model from earlier ###
y.shape
pred.shape

example_batch_loss = compute_loss( y, pred)

print(f"Prediction Shape: {pred.shape} #(Batch_size , Sequence_lenght , vocab_size)")
print(f"Scalar.loss : {example_batch_loss.mean().item()}")

In [ ]:
### HyperParameters Setting and Optimization ###
Vocal_size = len(vocab)

# Model Parameters
para = dict(
    num_training_iterations = 300,
     batch_size = 8,
     seq_length = 100,
     learning_rate = 5e-3,
     embedding_dim = 256,
     hidden_size = 1024
)

#CHeckout Points
Checkpoint_dir = './Training_checkpoints'
checkpoint_prefix = os.path.join(Checkpoint_dir , "my_ckpt")
os.makedirs(Checkpoint_dir , exist_ok=True )

In [ ]:
### CREATE a Comet Experiment to track Our training Experiment ###

def create_experiment ():
    # End any Prior Experiments
    if 'experiment' in locals():
        experiment.end()

    experiment = comet_ml.Experiment(
        api_key=comet_api_key ,
        project_name = "Genrating_Music" )

    # log our hyperparameters  , defined above , to the parameters
    for param , value  in para.items():
        experiment.log_parameter(param , value)
    experiment.flush()

    return experiment

In [ ]:
### Define optimizer and training operation ###
model = lstmmodel(vocab_size , embedding_dim , hidden_size)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)

def train_step(x,y):
    model.train()

    optimizer.zero_grad()

    y_hat = model(x)

    loss = compute_loss(y , y_hat)
    loss.backward()
    optimizer.step()

    return loss

In [ ]:
### TRAINING THE DATA STARTS ###
history = []
plotter = mdl.util.PeriodicPlotter(sec = 2 , xlabel='iterations', ylabel='loss')
experiment = create_experiment()

if hasattr(tqdm , '_instances'): tqdm._instances.clear()
for iter in tqdm(range(para["num_training_iterations"])):
    x_batch , y_batch = get_batch(vectorized_songs , para['seq_length'], para['batch_size'])

    x_batch = torch.tensor(x_batch , dtype=torch.long).to(device)
    y_batch = torch.tensor(y_batch , dtype=torch.long).to(device)

    loss = train_step(x_batch , y_batch)
    experiment.log_metric("loss" , loss.item(), step = iter)

    history.append(loss.item())
    plotter.plot(history)

    if iter % 100 == 0:
        torch.save(model.state_dict(),checkpoint_prefix)


    torch.save(model.state_dict,checkpoint_prefix)
    experiment.flush()

In [ ]:
def generate_text(model, start_string, char2idx, idx2char, device, generation_length=1000):
    model.eval()
    input_idx = [char2idx[char] for char in start_string]
    input_idx = torch.tensor([input_idx], dtype=torch.long).to(device)

    state = model.init_hidden(input_idx.size(0), device)
    text_generated = []

    for i in tqdm(range(generation_length)):
        predictions, state = model(input_idx, state, return_state=True)
        predictions = predictions.squeeze(0)
        input_idx = torch.multinomial(torch.softmax(predictions[-1], dim=-1), num_samples=1)
        text_generated.append(idx2char[input_idx.item()])
        input_idx = input_idx.unsqueeze(0)

    return start_string + ''.join(text_generated)

generated_text = generate_text(model, "X:", char2idx, idx2char, device)
#mdl.lab1.play_generated_song(generate_text)

In [ ]:
### Play back generated songs ###

generated_songs = mdl.lab1.extract_song_snippet(generated_text)

for i, song in enumerate(generated_songs):
  # Synthesize the waveform from a song
  waveform = mdl.lab1.play_song(song)

  # If its a valid song (correct syntax), lets play it!
  if waveform:
    print("Generated song", i)
    display.display(waveform)

    numeric_data = np.frombuffer(waveform.data, dtype=np.int16)
    wav_file_path = f"output_{i}.wav"
    write(wav_file_path, 88200, numeric_data)

    # save your song to the Comet interface -- you can access it there
    experiment.log_asset(wav_file_path)

In [ ]:
# when done, end the comet experiment
experiment.end()